In [33]:
#modules + packages
import os
import time
import math
import cv2 as cv
import statistics
import numpy as np
from ultralytics import YOLO

#colors:
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
yellow = (0, 255, 255)

#directories:
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
pt_path = os.path.join(project_root, os.path.join('models', 'yolo11m.pt'))
data_path = os.path.join(project_root, os.path.join('data', 'data.txt'))
video_path = os.path.join(project_root, os.path.join('UPLOAD VIDEO HERE', 'video.mp4'))

basketball_diameter = 0.24

#store needed data
ball_widths = [] #contains the ball widths in each frame, in order
ball_heights = [] #contains the ball heights in each frame, in order
person_heights = [] #contains the total vertical distance from top of body (usually head or hand) to the feet in each frame, in order
person_max_y = [] #contains the location of the top of the body (usually head or hand) in pixels in each frame, in order
person_max_y_valid_frames = [] #contains all person tops for only the frames the ball appears
frame_max_y_tuples = [] #lists the ordered pair (frame_num, person_max_y)
person_min_y = [] #contains the location of the feet in pixels in each frame, in order
ball_positions = [] #contains the location of center of ball in pixels in each frame, in order
ball_bottom = [] #stores only the bottom of the ball
valid_frames = [] #lists in order all of the frames in which the ball was detected

H = 3.048
a = -9.81

In [34]:
#Set up YOLO--------------
frame_angles_data = {}

ball = YOLO(pt_path)

cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Cannot open video.')

fps = float(cap.get(cv.CAP_PROP_FPS))
t_per_frame = 1/fps #finds the time elapsed between frames

start_time = time.time()
duration = 10

min_size = 10
min_person_size = 10
ball_color = green
wrist_color = blue 
person_color = red
#-----------------

#MAIN BALL DETECTION LOOP------------
try:
    #this just sets up a time constraint so that once enough data is collected, it stops
    while True:
        if (time.time() - start_time) > duration:
            break
            
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        ret, frame = cap.read()
        
        if not ret:
            break

        #updates frame number with each iteration
        frame_num = int(cap.get(cv.CAP_PROP_POS_FRAMES))

        #basketball and person position detecting-------
        basketballs = []
        results = ball(frame, verbose=False)
        
        for result in results:
            for box in result.boxes:
                if int(box.cls) == 32 and box.conf > 0.05: #ball class ID is 32
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    if w > min_size and h > min_size:
                        basketballs.append((x1, y1, x2, y2, box.conf.item()))

        persons = []
        for result in results:
            for box in result.boxes:
                if int(box.cls) == 0 and box.conf > 0.5: #person class ID is 0
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    w, h = x2 - x1, y2 - y1
                    if w > min_person_size and h > min_person_size:
                        persons.append((x1, y1, x2, y2, box.conf.item()))
        
        if persons:
            persons.sort(key=lambda p: p[4], reverse=True)
            best_person = persons[0]
            x1, y1, x2, y2, conf = best_person
            person_heights.append(y2-y1) #stores person's total vertical height in each frame
            person_max_y.append(432-y1) #stores the highest point of the person in each frame
            person_min_y.append(432-y2) #stores the lowest point of the person in each frame
            frame_max_y_tuples.append((frame_num, 432-y1)) #stores the highest point of the person in each from along with the frame number
            cv.rectangle(frame, (x1, y1), (x2, y2), person_color, 2) #draws rectangle around person
            cv.putText(frame, f"Person ({conf:.2f})", (x1, y1-10),
                      cv.FONT_HERSHEY_SIMPLEX, 0.7, person_color, 2)
        
        best_ball = None
        if basketballs:
            basketballs.sort(key=lambda b: b[4], reverse=True)
            best_ball = basketballs[0]

        ball_dimensions = None
        if best_ball:
            x1, y1, x2, y2, conf = best_ball
            current_ball_pos = ((x1 + x2) / 2, ((432-y2) + (432-y1)) / 2)
            
            ball_widths.append(x2-x1) #stores ball widths
            ball_heights.append((432-y1)-(432-y2)) #stores ball heights
            ball_positions.append(current_ball_pos) #stores the ordered pair of the current ball position
            ball_bottom.append((432-y2)) #stores the lowest point of the ball
            valid_frames.append(frame_num) #stores the current frame as a frame in which YOLO successfully detected the ball

        if best_ball:
            x1, y1, x2, y2, conf = best_ball
            cv.rectangle(frame, (x1, y1), (x2, y2), ball_color, 1) #draws rectangle around ball
            cv.putText(frame, f"Basketball ({conf:.2f})", (x1, y1-10), 
                       cv.FONT_HERSHEY_SIMPLEX, 0.7, ball_color, 2)
        #-------
        
        status = "Ball: " + ("Detected" if best_ball else "Missing")
        cv.putText(frame, status, (10, frame.shape[0] - 20), 
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, white, 2)
        
        cv.imshow("Basketball Shot Analysis", frame) #displays the video analysis
       
        #this is for this video specifically, will be changed later
        if frame_num > 15:
            break
            
except Exception as e:
    import traceback
    traceback.print_exc()
finally:
    cap.release()
    cv.destroyAllWindows()

In [35]:
#CODE CELL 3: calculations

#A "valid frame" is one in which YOLO successfully detects the ball.

ball_width_px = statistics.median(ball_widths)
ball_height_px = statistics.median(ball_heights)
avg_dimension = ball_width_px #finds ball's dimensions in px

cf = basketball_diameter/avg_dimension #1px = this number of meters; this is the conversion factor


min_height = min(person_min_y) 
#The absolute minimum of all the lowest positions of the person in the frames is ground level (because the person's feet can't get any lower).


#construct person_max_y_valid_frames list
for i in range (0, len(valid_frames)):
    target = frame_max_y_tuples[valid_frames[i]-1] #finds all the tuples corresponding to a frame in which basketball was detected
    person_max_y_valid_frames.append(target[1]) #appends the second term (the height) to person_max_y_valid_frames

#find the frame of release
for i in range (0, len(valid_frames)):
    #search for the two frames between which the ball's bottom rises above the hand
    if ball_bottom[i] > person_max_y_valid_frames[i] and ball_bottom[i-1] <= person_max_y_valid_frames[i-1]:
        frame_of_release = valid_frames[i-1] #finds the last frame in which the ball was below hand level and sets it as frame of release
        ind = i-1 #also returns the index
        break

launch_height_tuple = ball_positions[ind] #finds the ball's position at the frame of release
launch_height = (launch_height_tuple[1]-min_height)*cf #gets the y-coordinate of that position
launch_x = (launch_height_tuple[0])*cf #gets the x-coordinate of that position

#with open(data_path, 'w') as f:
    #f.write(str(launch_height))

#for loop to determine the least valid frame greater than the frame of release
for i in range (0, len(valid_frames)):
    if valid_frames[i] > frame_of_release:
        least_valid_frame = valid_frames[i]
        break
    else:
        least_valid_frame = None


ball_height_lvf_tuple = ball_positions[valid_frames.index(least_valid_frame)] #gets ball's position at the next valid frame
#this is the ball height at the "least_valid_frame"
ball_height_lvf = (ball_height_lvf_tuple[1]-min_height)*cf #ball's height
ball_x_lvf = (ball_height_lvf_tuple[0])*cf #ball's horizontal distance


vertical_disp = ball_height_lvf - launch_height

horizontal_disp = ball_x_lvf - launch_x
elapsed_time = (least_valid_frame - frame_of_release-0.1)*(t_per_frame) #the -0.1 takes into account that at the moment of the launch as previously written, technically, the ball is still in the hand. This tiny amount makes a huge difference.
#****This value of -0.1 was determined empirically. For the purposes of this demo, this value will make the program work, at least with the specific provided video.******

initial_vel_up = (vertical_disp - 0.5*(a)*(elapsed_time)**2)/elapsed_time #see below for the kinematics equation used

vel_horizontal = horizontal_disp/elapsed_time

#*****


$$\Delta y = v_{0, y} t + \frac{1}{2} a t^2$$

In [36]:
h = launch_height
v_0 = np.sqrt((initial_vel_up)**2 + (vel_horizontal)**2)
dist = 4.191 #for now, assume a shot from the free throw line
detected_angle = np.arctan(initial_vel_up/vel_horizontal)

#

diff_h = H - h

#****
#See below for an explanation on how the optimal angle is calculated.
#****



#kinematics equations can be used to solve for z = tanx, where x is the unknown angle in radians. In this quadratic equation:
A = 0.5*a*((dist**2)/((v_0**2)))
B = dist
C = 0.5*a*((dist**2)/((v_0**2)))-(diff_h)

real_solutions = False

def rad_to_deg(angle):
    return(angle*180/np.pi)

if B**2-4*A*C < 0:
    print('You need to shoot faster! You shot at ' + str(round(v_0,2)) + 'm/s.') #if no real solutions, then that means launch speed was insufficient and the player missed
else:
    real_solutions = True
    
def quadratic_formula(a, b, c):
        pos_solution = (-b-np.sqrt(b**2-4*a*c))/(2*a)
        real_solutions = True
        return pos_solution #In a normal situation, z = tanx should be positive, so only the positive solution is returned. It can be proven that the second solution must always be negative.

if real_solutions == True:
    z = float(quadratic_formula(A, B, C)) #solves for optimal launch angle
    angle_rad = np.arctan(z)

    theta = round(rad_to_deg(angle_rad), 2)

    #output
    print('The optimal launch angle is ' + str(theta) + ' degrees.')

    #I set a margin of error of 3 degrees. This could be improved later
    if theta > rad_to_deg(detected_angle) and abs(theta - rad_to_deg(detected_angle)) > 3:
        if abs(theta - rad_to_deg(detected_angle)) <= 8:
            print('Try shooting a bit higher, and/or shooting slower. You shot at ' + str(round(rad_to_deg(detected_angle),2)) + ' degrees.')
            print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s.')
        else:
            print('Try shooting quite a bit higher, and/or shooting slower. You shot at ' + str(round(rad_to_deg(detected_angle),2)) + ' degrees.')
            print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s.')
    elif theta < rad_to_deg(detected_angle) and abs(theta - rad_to_deg(detected_angle)) > 3:
        if abs(theta - rad_to_deg(detected_angle)) <= 8:
            print('Try shooting a bit lower, and/or shooting faster. You shot at ' + str(round(rad_to_deg(detected_angle),2)) + ' degrees.')
            print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s.')
        else:
            print('Try shooting quite a bit lower, and/or shooting faster. You shot at ' + str(round(rad_to_deg(detected_angle),2)) + ' degrees.')
            print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s.')
    else:
        print("You shot at " + str(round(rad_to_deg(detected_angle),2)) + " degrees. You made a basket (within this program's margin of error)!")
        print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s.')
elif real_solutions == False:
        print('Your shooting speed was ' + str(round(v_0,2)) + 'm/s, and you shot at ' + str(round(rad_to_deg(detected_angle),2)) + ' degrees.')

The optimal launch angle is 56.34 degrees.
You shot at 54.7 degrees. You made a basket (within this program's margin of error)!
Your shooting speed was 7.28m/s.


# Calculating the Theoretical Optimal Launch Angle
We are given the following:
$$\Delta y, a, \Delta x, v_0$$
In order to solve for $\theta$, we will try to derive a quadratic equation in terms of $\tan\theta$ and then use the quadratic formula.
Next, use kinematics equations. On the x-axis and y-axis, respectively:
$$\Delta x = v_{0, x} t = \left(v_0 cos \theta\right)t$$
$$\Delta y = v_{0, y} t + \frac{1}{2} a t^2 = \Delta y = \left(v_0 sin \theta\right) t + \frac{1}{2} a t^2$$
Also, the following trigonometric identity will be used:
$$sec^2\theta = tan^2\theta + 1$$
Now, we can derive:
$$\Delta x = v_{0, y} t + \frac{1}{2} a t^2 = \Delta y = \left(v_0 sin \theta\right) \left(\frac{\Delta x}{v_0 cos \theta}\right) + \frac{1}{2} a \left(\frac{\Delta x}{v_0 cos \theta}\right)^2 = \Delta x tan\theta + \frac{1}{2} a \left(\frac{\Delta x}{v_0 cos \theta}\right)^2$$
Cleaning up, continuing where we left off, and using the aforementioned identity:
$$\Delta y = \Delta x tan\theta + \frac{1}{2} a \frac{\left(\Delta x\right)^2}{\left(v_0\right)^2}sec^2\theta = \Delta x tan\theta + \frac{1}{2} a \frac{\left(\Delta x\right)^2}{\left(v_0\right)^2}tan^2\theta + \frac{1}{2} a \frac{\left(\Delta x\right)^2}{\left(v_0\right)^2}$$
Rearranging:
$$\frac{1}{2} a \frac{\left(\Delta x\right)^2}{\left(v_0\right)^2}tan^2\theta + \Delta x tan\theta + \left(\frac{1}{2} a \frac{\left(\Delta x\right)^2}{\left(v_0\right)^2} - \Delta y\right) = 0$$
Now, the quadratic formula can be used to find $tan\theta$, and then $\theta$ is easily found.